<a href="https://colab.research.google.com/github/grant88/education/blob/main/spark_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 28 kB/s 
     |████████████████████████████████| 198 kB 43.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=004d65f53ec22730bc04e7e086d74a090f1b8dc0219dd2db1cf593d759f73ee1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window

In [11]:
spark = SparkSession.builder.master("local[*]").appName("PySpark").getOrCreate()

In [12]:
df = spark.read.csv("owid-covid-data.csv", header=True, inferSchema=True)

### Задание 1
15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [14]:
(
    df
    .filter(
        (f.col("date") == "2021-03-31")
        & (f.length(f.col("iso_code")) == 3)
    )
    .select(
        "iso_code",
        "location",
        f.round(100 * f.col("total_cases") / f.col("population"), 2).alias("ill_percent")
    )
    .sort(f.col("ill_percent").desc())
    .show(15)
)

+--------+-------------+-----------+
|iso_code|     location|ill_percent|
+--------+-------------+-----------+
|     AND|      Andorra|      15.54|
|     MNE|   Montenegro|      14.52|
|     CZE|      Czechia|      14.31|
|     SMR|   San Marino|      13.94|
|     SVN|     Slovenia|      10.37|
|     LUX|   Luxembourg|       9.85|
|     ISR|       Israel|       9.63|
|     USA|United States|        9.2|
|     SRB|       Serbia|       8.83|
|     BHR|      Bahrain|       8.49|
|     PAN|       Panama|       8.23|
|     PRT|     Portugal|       8.06|
|     EST|      Estonia|       8.02|
|     SWE|       Sweden|       7.97|
|     LTU|    Lithuania|       7.94|
+--------+-------------+-----------+
only showing top 15 rows



### Задание 2
Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [15]:
window = Window.partitionBy("iso_code")

(
    df
    .where(
        (df.date >= "2021-03-25") 
        & (df.date <= "2021-03-31")
        & (f.length(df.iso_code) == 3)
    )
    .withColumn("max_cases_week", f.max("new_cases").over(window))
    .where(f.col("max_cases_week") == f.col("new_cases"))
    .sort(f.col("max_cases_week").desc())
    .select(
        "date",
        "location",
        "new_cases"
    )
    .show(10)
)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-03-25|       Brazil| 100158.0|
|2021-03-26|United States|  77321.0|
|2021-03-31|        India|  72330.0|
|2021-03-31|       France|  59054.0|
|2021-03-31|       Turkey|  39302.0|
|2021-03-26|       Poland|  35145.0|
|2021-03-31|      Germany|  25014.0|
|2021-03-26|        Italy|  24076.0|
|2021-03-25|         Peru|  19206.0|
|2021-03-26|      Ukraine|  18226.0|
+----------+-------------+---------+
only showing top 10 rows



### Задание 3
Изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021.

(например: в россии вчера было 9150 , сегодня 8763, итог: -387)

(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [16]:
window = Window.partitionBy("iso_code").orderBy(f.col("date"))

(
    df
    .where(
        (df.date >= "2021-03-24") 
        & (df.date <= "2021-03-31")
        & (df.iso_code == 'RUS')
    )
    .withColumn("yesterday_new_cases", f.lag("new_cases", 1).over(window))
    .select(
        "date",
        "yesterday_new_cases",
        "new_cases",
        (f.col("new_cases") - f.col("yesterday_new_cases")).alias("delta")
    )
    .orderBy(f.col("date").desc())
    .limit(7)
    .orderBy(f.col("date"))
    .show()
)

+----------+-------------------+---------+------+
|      date|yesterday_new_cases|new_cases| delta|
+----------+-------------------+---------+------+
|2021-03-25|             8769.0|   9128.0| 359.0|
|2021-03-26|             9128.0|   9073.0| -55.0|
|2021-03-27|             9073.0|   8783.0|-290.0|
|2021-03-28|             8783.0|   8979.0| 196.0|
|2021-03-29|             8979.0|   8589.0|-390.0|
|2021-03-30|             8589.0|   8162.0|-427.0|
|2021-03-31|             8162.0|   8156.0|  -6.0|
+----------+-------------------+---------+------+

